In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.2`

import $ivy.$                                  

In [2]:
import $ivy.`io.delta::delta-core:2.0.0`

import $ivy.$                           

In [3]:
import io.delta._
import io.delta.tables._
import org.apache.spark.sql.functions._

import io.delta._

import io.delta.tables._

import org.apache.spark.sql.functions._

In [4]:
val spark = org.apache.spark.sql.SparkSession.builder()
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .appName("test").master("local").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/08/08 12:33:34 INFO SparkContext: Running Spark version 3.2.2
22/08/08 12:33:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/08 12:33:34 INFO ResourceUtils: ==============================================================
22/08/08 12:33:34 INFO ResourceUtils: No custom resources configured for spark.driver.
22/08/08 12:33:34 INFO ResourceUtils: ==============================================================
22/08/08 12:33:34 INFO SparkContext: Submitted application: test
22/08/08 12:33:34 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
22/08/08 12:33:34 INFO ResourceProfile:

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@74ec78ee

In [5]:
val data = spark.range(0, 5)
data.write.format("delta").save("tmp/delta-table")

data: org.apache.spark.sql.Dataset[java.lang.Long] = [id: bigint]

## Read data

In [7]:
val df = spark.read.format("delta").load("tmp/delta-table")
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



df: org.apache.spark.sql.package.DataFrame = [id: bigint]

## Update table data: overwrite

In [8]:
val data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("tmp/delta-table")

data: org.apache.spark.sql.Dataset[java.lang.Long] = [id: bigint]

In [9]:
df.show()

+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



## Update table data: conditional update without overwrite

In [10]:
val deltaTable = DeltaTable.forPath("tmp/delta-table")

deltaTable: DeltaTable = io.delta.tables.DeltaTable@37b9b9ab

Update every even value by adding 100 to it

In [11]:
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = Map("id" -> expr("id + 100")))

In [12]:
deltaTable.toDF.show()

+---+
| id|
+---+
|  5|
|106|
|  7|
|108|
|  9|
+---+



Delete every even value

In [13]:
deltaTable.delete(condition = expr("id % 2 == 0"))

In [14]:
deltaTable.toDF.show()

+---+
| id|
+---+
|  5|
|  7|
|  9|
+---+



Upsert (merge) new data

In [15]:
val newData = spark.range(0, 20).toDF

deltaTable.as("oldData")
  .merge(
    newData.as("newData"),
    "oldData.id = newData.id")
  .whenMatched
  .update(Map("id" -> col("newData.id")))
  .whenNotMatched
  .insert(Map("id" -> col("newData.id")))
  .execute()

newData: org.apache.spark.sql.package.DataFrame = [id: bigint]

In [16]:
deltaTable.toDF.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



## Read older versions of data with time travel

In [17]:
val df = spark.read.format("delta").option("versionAsOf", 0).load("tmp/delta-table")

df: org.apache.spark.sql.package.DataFrame = [id: bigint]

In [18]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

